Evitando sobreajuste por meio de regularização

In [6]:
# Python > 3.5 é necessario
import sys
assert sys.version_info >= (3, 5)

# Scikit-learn >0.20 é necessário
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version 2.x  # Este comando é usado apenas no Google Colab
    pass
except Exception:
    pass


# Tensorflow >2.0 é necessário
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

#%load_ext tensorboard

import numpy as np
import os

np.random.seed(42)


In [7]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

pixel_mens = X_train.mean(axis=0, keepdims = True)
pixel_stds = X_train.std(axis=0, keepdims = True)
X_train_scaled = (X_train - pixel_mens) / pixel_stds
X_valid_scaled = (X_valid - pixel_mens) / pixel_stds
X_test_scaled = (X_test - pixel_mens) / pixel_stds

Regularização pela tecnica Dropout

In [8]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
                metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                      validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7581 - loss: 0.7885 - val_accuracy: 0.8652 - val_loss: 0.4161
Epoch 2/2
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8369 - loss: 0.4734 - val_accuracy: 0.8754 - val_loss: 0.3599


Tecnica Alpha Dropout

In [9]:
tf.random.set_seed(42)
np.random.seed(42)

In [11]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X_train_scaled, y_train, epochs=20,validation_data=(X_valid_scaled, y_valid))

Epoch 1/20


TypeError: Exception encountered when calling AlphaDropout.call().

[1mgreater_equal() got an unexpected keyword argument 'seed'[0m

Arguments received by AlphaDropout.call():
  • inputs=tf.Tensor(shape=(None, 784), dtype=float32)
  • training=True